# YOLOv5 on SageMaker--推理

## 1 说明
本章内容为使用SageMaker进行推理，模型来自上一章的训练结果

## 2 运行环境
Kernel 选择pytorch_latest_p36。  
本文在boto3 1.17.12和sagemaker 2.26.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

## 3 获取image

本项目已build完毕image，存放到ECR中，可直接部署到SageMaker。请选择选择合适版本。

In [ ]:
tag = "v3.0"

In [ ]:
import boto3
region = boto3.session.Session().region_name
image_uri = '048912060910.dkr.ecr.{}.amazonaws.com.cn/nwcd/yolov5-inference:{}'.format(region,tag)
image_uri

## 4 部署模型到SageMaker

In [19]:
#读取model_data位置
model_data_file = "../1-training/model_data.txt"
if os.path.isfile(model_data_file):
    with open(model_data_file) as f:
        model_data = f.read()
        print(model_data)
else:
    print("未读取到model_data，请确认已完成训练")

In [ ]:
from sagemaker.model import Model
import boto3

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题
print(role)

endpoint_name = "yolov5"
my_model = Model(
            image_uri=image_uri,
            model_data=model_data,
            role=role,)

#该步骤大概需要15分钟
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.t2.medium'
                                )

## 5 推理

### 5.1 准备测试图片

In [ ]:
#请修改测试图片地址
data = {"bucket":"junzhong","image_uri":"yolov5/training/images/val/000729.jpeg"}

### 5.2 发送推理请求

返回的结果是对象的中心点位置(x,y)，以及宽高(w,h)

#### 方式一、使用boto3

In [ ]:
import boto3
import json
endpoint_name = "yolov5"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data),
                                  ContentType="application/json")
#print(response)
response_body = response['Body'] 
body= response_body.read()
result = json.loads(body.decode('utf-8'))
print(result)

#### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
result = predictor.predict(data)
print(result)

## 6 使用结果

### 6.1 定义函数

In [ ]:
!pip install tqdm -i https://pypi.tuna.tsinghua.edu.cn/simple/

In [ ]:
import sys
sys.path.append("source")
from IPython.display import Image
from utils.datasets import LoadImages
from utils.general import (xywh2xyxy, plot_one_box)
import cv2
from numpy import random
import torch
def xywh2xyxy(xywh,width,height):
    xyxy = []
    xyxy.append((xywh[0]-xywh[2]/2)*width)
    xyxy.append((xywh[1]-xywh[3]/2)*height)
    xyxy.append((xywh[0]+xywh[2]/2)*width)
    xyxy.append((xywh[1]+xywh[3]/2)*height)
    return xyxy

def draw(result,source_img,output_img):
    dataset = LoadImages(source_img)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(50)]
    for path, img, im0s, vid_cap in dataset:
        shape = torch.tensor(im0s.shape)
        for item in result:
            xyxy_list = xywh2xyxy(item["xywh"],int(shape[1]),int(shape[0]))
            xyxy = []
            for xyxy_item in xyxy_list:
                xyxy.append(torch.Tensor([xyxy_item]))
            label = '%s %.2f' % (item["class_name"], item["confidence"])
            plot_one_box(xyxy, im0s, label=label, color=colors[item["class"]], line_thickness=3)
    cv2.imwrite(output_img, im0s)

### 6.2 下载原图并显示

In [ ]:
import os
if not os.path.exists("result"):
    os.mkdir("result")

In [ ]:
s3_file="s3://"+data["bucket"]+"/"+data["image_uri"]
!aws s3 cp $s3_file ./result
source_file_name=data["image_uri"].split("/")[-1:][0]
source_img="result/"+source_file_name
Image(filename=source_img, width=600)

### 6.3 把推理结果显示在图片上

In [ ]:
output_img="result/output_"+source_file_name
draw(result,source_img,output_img)
Image(filename=output_img, width=600)

## 7 删除Endpoint

In [13]:
import boto3
endpoint_name = "yolov5"
sage = boto3.Session().client(service_name='sagemaker') 
sage.delete_endpoint(EndpointName=endpoint_name)
sage.delete_endpoint_config(EndpointConfigName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'ad409dfa-e137-4a57-9b87-b9d02ddd8d1e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ad409dfa-e137-4a57-9b87-b9d02ddd8d1e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Thu, 27 May 2021 12:43:36 GMT'},
  'RetryAttempts': 0}}